In [2]:
import pandas as pd
from datasets import load_dataset, Dataset
import os
import re
import torch
from sklearn.model_selection import train_test_split
import requests
import random


### PRE-PROCESSING

In [3]:
#si crea un dataframe con una riga per frase, attributi: id, testo e indice di gulpease
ds_df = pd.read_csv("ds_leggibilita.csv")
ds_df.head()

,Unnamed: 0,id,text,readability
0,1,2,La posizione di Bandiera Rossa.,17.622770
1,2,3,"Non mancò di pubblicare una raccolta di rime, ...",88.351943
2,3,4,"Considerava Peter come un figlio, sotto le cui...",86.028765
3,4,5,"Sfortunatamente, tra il 1997 e il giugno 2001,...",85.779941
4,6,7,"Terminata la prima guerra mondiale, che aveva ...",99.222837


In [4]:
ds_df.drop(columns=["Unnamed: 0"])

,id,text,readability
0,2,La posizione di Bandiera Rossa.,17.622770
1,3,"Non mancò di pubblicare una raccolta di rime, ...",88.351943
2,4,"Considerava Peter come un figlio, sotto le cui...",86.028765
3,5,"Sfortunatamente, tra il 1997 e il giugno 2001,...",85.779941
4,7,"Terminata la prima guerra mondiale, che aveva ...",99.222837
...,...,...,...
7672,9995,"""La Ricciarda"" (tragedia lirica;",90.633286
7673,9996,"All'interno di una coppia, nella quale entramb...",74.484438
7674,9997,Enric Morera fu il musicista più dotato per il...,30.300036
7675,9998,"""Chiu-ling Shan"") sono una catena montuosa sit...",45.706033


In [5]:
#Divido in training e test set
dataset = Dataset.from_pandas(ds_df)
split_set = dataset.train_test_split(test_size=0.1)

train_ds = split_set["train"]
test_ds = split_set["test"]

### TOKENIZATION

In questa sezione si importa il tokenizzatore col quale si tokenizza ciascuna frase nel formato necessario per Bert, alla fine si otterrà un dataset nel formato corretto con tutte le features necessarie per il training

In [6]:
import tokenizers
import transformers
from transformers import BertTokenizer

In [7]:
#si importa il tokenizzatore già configurato (in questo caso: bert-base-italian-cased)
tokenizer = BertTokenizer.from_pretrained("dbmdz/bert-base-italian-cased")


In [8]:
#facciamo l'encoding di tutto il dataset tokenizzando frase per frase
def encode(sample):
    return tokenizer(sample["text"], padding=True, truncation=True, max_length=512, return_special_tokens_mask=True)

train_set = train_ds.map(encode, batched=True)
test_set = test_ds.map(encode, batched=True)
train_set.set_format('torch', columns=["input_ids", "attention_mask", "token_type_ids"])
test_set.set_format('torch', columns=["input_ids", "attention_mask", "token_type_ids"])


Map:   0%|          | 0/6909 [00:00<?, ? examples/s]

Map:   0%|          | 0/768 [00:00<?, ? examples/s]

In [9]:
train_set

Dataset({
    features: ['Unnamed: 0', 'id', 'text', 'readability', 'input_ids', 'token_type_ids', 'special_tokens_mask', 'attention_mask'],
    num_rows: 6909
})

In [10]:
test_set

Dataset({
    features: ['Unnamed: 0', 'id', 'text', 'readability', 'input_ids', 'token_type_ids', 'special_tokens_mask', 'attention_mask'],
    num_rows: 768
})

### TRAINING DI BERT

Si procede al training di Bert. Il modello dovrà partire da uno stato iniziale con pesi random, per questo non si importa il modello già addestrato, ma si configura semplicemente l'architettura la sua architettura per poi addestrarlo da zero. Si definisce poi una strategia di training e i suoi argomenti per poi addestrare il modello sul trask di Language Modeling. 

In [11]:
from transformers import Trainer, TrainingArguments, TrainerCallback, BertConfig, BertForMaskedLM, DataCollatorForLanguageModeling, set_seed


In [12]:
model_name = "prajjwal1/bert-mini"
model_config = BertConfig.from_pretrained(model_name)

print(model_config)

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 4,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [13]:
model = BertForMaskedLM(model_config)
model.resize_token_embeddings(len(tokenizer))


Embedding(31102, 256)

In [14]:
model.config

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 4,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 31102
}

In [15]:
#usiamo il datacollator per fare le batch per il training
datacollator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.2, return_tensors="pt")

In [16]:
datacollator

DataCollatorForLanguageModeling(tokenizer=PreTrainedTokenizer(name_or_path='dbmdz/bert-base-italian-cased', vocab_size=31102, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}), mlm=True, mlm_probability=0.2, pad_to_multiple_of=None, tf_experimental_compile=False, return_tensors='pt')

In [17]:
print(f"Lunghezza del dataset: {len(train_set)}")

Lunghezza del dataset: 6909


In [18]:
#definisco una funzione di callback per verificare l'ordinamento dei dati per ogni epoca
from transformers.trainer_callback import TrainerControl, TrainerState
from transformers.training_args import TrainingArguments


class check_ds_order(TrainerCallback):
    def on_epoch_begin(self, args, state, control, train_dataloader, **kwargs):
        f = open(f"train_check.txt", "a")
        f.write(f"\n------------------------ ORDINE DEI DATI ALL'INIZI DELL'EPOCA {int(state.epoch+1)} ------------------------")
        f.write(str(train_dataloader.dataset["input_ids"][:5]))
        f.write("-----------------------------------------------------------------------------------")

class check_weights(TrainerCallback):
    def on_train_begin(self, args, state, control, model, **kwargs):
        init_weights = model.state_dict()
        for key, value in init_weights.items():
            print(f"\n{key}:\n")
            print(value)


In [19]:
#argomenti provvisori, da definire meglio
training_args = TrainingArguments(
    output_dir = "my_pretrained_model",
    evaluation_strategy="steps",
    overwrite_output_dir=True,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=128,
    num_train_epochs=3,
    logging_steps=10,
    save_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True,
    seed=42, 
    )

In [20]:
set_seed(training_args.seed)

In [21]:
open('train_check.txt', 'w').close()

In [22]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=datacollator,
    train_dataset=train_set,
    eval_dataset=test_set,
    callbacks=[check_ds_order, check_weights], 
    )

In [23]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask, id, text, readability, Unnamed: 0. If special_tokens_mask, id, text, readability, Unnamed: 0 are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
c:\Users\bergo\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 6909
  Num Epochs = 3
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 324



bert.embeddings.position_ids:

tensor([[  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
          14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
          28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
          42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
          56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
          70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
          84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
          98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
         112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
         126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
         140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
         154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
         168, 169, 1

  0%|          | 0/324 [00:00<?, ?it/s]